In [111]:
import dadi, pickle
import ipywidgets as widgets
from ipywidgets import FloatSlider, interact_manual, interact, HBox, Tab, Button, ButtonStyle, Layout
from IPython.display import display
from Models import get_dadi_model_func, get_dadi_model_params

Please upload your frequency spectrum data (in dadi's frequency spectrum file format), select grid point settings used in extrapolation during model inference, select the demographic model you are using, and choose whether you will inlcude ancestral state misidentification in the model, i.e., is the frequency spectrum folded or unfolded?

In [112]:
# Code that sets up interactive widgets, so that users can input their desired values

# Upload data widget
upload = widgets.FileUpload(
               accept='',  # Accepted file extension e.g. '.fs'
               multiple=False  # True to accept multiple files upload else False
           )

#Select grid size widget
a = widgets.BoundedIntText(
    value=40,
    min=1,
    max=10000,
    step=1,
    description='Grid-x:',
    disabled=False
)
b = widgets.BoundedIntText(
    value=50,
    min=1,
    max=10000,
    step=1,
    description='Grid-y:',
    disabled=False
)
c = widgets.BoundedIntText(
    value=60,
    min=1,
    max=10000,
    step=1,
    description='Grid-z:',
    disabled=False
)

grid_ui = widgets.HBox([a, b, c])


#Select demographic model widget
model = widgets.Dropdown(
    options=['bottlegrowth_1d', 'growth_1d', 'snm_1d', 'three_epoch_1d', 'two_epoch_1d',
             'bottlegrowth_2d', 'bottlegrowth_split', 'bottlegrowth_split_mig', 'IM', 'IM_pre', 'snm_2d', 'split_mig', 'split_asym_mig'],
    description='Model:',
)

#Choose misid status widget
misid = widgets.Dropdown(
    options=['True', 'False'],
    description='Misid:',
)

# layout widgets together 
tab = widgets.Tab()
tab_contents = ['File upload', 'Grid points', 'Model', 'Misidentification']
tab.children = [upload, grid_ui, model, misid]
for title, (index, _) in zip(tab_contents, enumerate(tab.children)):
    tab.set_title(index, title)
tab


In [113]:
# Load the data from user input
uploaded_filename = next(iter(upload.value))
content = upload.value[uploaded_filename]['content']
with open(uploaded_filename, 'wb') as f: f.write(content)
# Read data into dadi
data = dadi.Spectrum.from_file(uploaded_filename)
ns = data.sample_sizes

# These are the grid point settings will use for extrapolation.
pts_l = [a.value, b.value, c.value]

# Make a demographic model function, using the model provided by the user.
func = get_dadi_model_func(model.value)
params = get_dadi_model_params(model.value)

# If the misid option is chosen, we update our model to include this, using dadi's built-in function 
if misid.value == 'True':
    func = dadi.Numerics.make_anc_state_misid_func(func)
    params = get_dadi_model_params(model.value)
    params.append('misid')

# Make the extrapolating version of our demographic model function.
func_ex = dadi.Numerics.make_extrap_log_func(func)



StopIteration: 

Select upper and lower bounds for each of the demographic model parameters (or use the default values provided here).

In [118]:
# upper bounds sliders
upper_sliders = []
for p in params:
    if p == 's':
        s = widgets.BoundedFloatText(
            value=1,
            min=0,
            max=1,
            step=0.1,
            description=p+':',
        )
    else:
        s = widgets.BoundedFloatText(
                value=10,
                min=0,
                max=10000,
                step=0.1,
                description=p+':',
        )
    upper_sliders.append(s)
    
upper_ui = widgets.VBox(upper_sliders)

# lower bounds sliders
lower_sliders = []
for p in params:
    if p == 's':
        s = widgets.BoundedFloatText(
            value=0.1,
            min=0,
            max=1,
            step=0.1,
            description=p+':',
        )
    else:
        s = widgets.BoundedFloatText(
                value=0.1,
                min=0,
                max=10000,
                step=0.1,
                description=p+':',
        )
    lower_sliders.append(s)
    
lower_ui = widgets.VBox(lower_sliders)

# layout upper and lower bouunds
header  = Button(description='Lower bounds',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
lower_ui = widgets.VBox([header, lower_ui])

header  = Button(description='Upper bounds',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
upper_ui = widgets.VBox([header, upper_ui])


widgets.HBox([lower_ui, upper_ui])


In [119]:
# read in the lower and upper bounds- these will provide the limits for the p0 sliders
lower_bound = []
for s in lower_sliders:
    lower_bound.append(s.value)

upper_bound = []
for s in upper_sliders:
    upper_bound.append(s.value)


You can now use the sliders and button below to try out initial parameter values (p0) to fit a demographic model to your data. For details on the units used by dadi for parameter values, please see the manual (https://dadi.readthedocs.io/en/latest/user-guide/specifying-a-model/). You can reset the ranges of any of the parameter sliders by changing their upper and lower bounds in the widget above.

In [120]:
def plot_func_2d(**sliders_dict):    
    p0 = [x for x in sliders_dict.values()]
    print('Here are the model parameters ('+model.value+') used to generate this plot:')
    print(p0)
    test_model = func_ex(p0, ns, pts_l)
    dadi.Plotting.plot_2d_comp_multinom(test_model, data, vmin=1, resid_range=3, show=True)

def plot_func_1d(**sliders_dict):
    p0 = [x for x in sliders_dict.values()]
    print('Here are the model parameters ('+model.value+') used to generate this plot:')
    print(p0)
    test_model = func_ex(p0, ns, pts_l)
    dadi.Plotting.plot_1d_comp_multinom(test_model, data, vmin=1, resid_range=3, show=True)

    
sliders_dict = {}
for p, l, u in zip(params, lower_bound, upper_bound):
    sliders_dict[p+':'] = FloatSlider(
        value=0.5,
        min=l,
        max=u,
        step=0.1,
    )

if '1d' not in model.value:
    interact_manual(plot_func_2d, **sliders_dict)
else:
    interact_manual(plot_func_1d, **sliders_dict)
    

interactive(children=(FloatSlider(value=0.5, description='nuPre:', max=10.0, min=0.1), FloatSlider(value=0.5, …

In [ ]:
def save_params(Filename):
    p0 = [sliders_dict[p+':'].value for p in params]
    dic = {
        'params':p0,
        'demographic_model':func,
        'sample_sizes':ns,
        'gird_points':pts_l,
        'lower_bound':lower_bound,
        'upper_bound':upper_bound,
    }
    print("Saving demographic model information as: "+Filename)
    fid=open(Filename,'wb')
    pickle.dump(dic, fid, 2)
    fid.close()
print('What file name do you want to save you demographic model information under?')
interact_manual(save_params, Filename='demo_params.bpkl')